In [1]:
import sys
import datetime
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
os.chdir("/mnt/c/Users/Dave/Project/COBS")
from cobs import Model, Reward
from cobs import OccupancyGenerator as OG
from cobs.predictive_model.pkl_importer import pklImporter
from tqdm import tqdm
from pprint import pprint

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from config import state_names, disturbances_dict, eplus_naming_dict, eplus_var_types, SatAction
from agents.DQNAgent import *
from agents.Networks.DeepQ import *
from agents.BDDQNAgent import *
from agents.Networks.BDDQN import *
from utils.agent import *
from main import *

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
dist_names = list(disturbances_dict.values())
dist_names

['Ambient Temp.',
 'Outdoor RH',
 'Wind Speed',
 'Wind Direction',
 'Diffuse Solar Rad.',
 'Direct Solar Rad.']

## Load E+ simulation result

In [4]:
# Set Pre E+ simulation results
dataset_name = "simulation_results/Sim-chicago-summer.pkl"
obs_history = pd.read_pickle(dataset_name)
disturbance = obs_history[dist_names]

# Min-Max Normalization
disturbance = (disturbance - disturbance.min())/(disturbance.max() - disturbance.min())
all_exp_time = disturbance.index
disturbance

,Ambient Temp.,Outdoor RH,Wind Speed,Wind Direction,Diffuse Solar Rad.,Direct Solar Rad.
time,,,,,,
1991-07-01 00:15:00,0.258294,0.832031,0.275424,0.972028,0.0,0.0
1991-07-01 00:30:00,0.255924,0.820312,0.288136,0.041958,0.0,0.0
1991-07-01 00:45:00,0.253555,0.808594,0.300847,0.118881,0.0,0.0
1991-07-01 01:00:00,0.251185,0.796875,0.313559,0.195804,0.0,0.0
1991-07-01 01:15:00,0.247630,0.796875,0.315678,0.209790,0.0,0.0
...,...,...,...,...,...,...
1991-07-15 23:00:00,0.658768,0.859375,0.347458,0.727273,0.0,0.0
1991-07-15 23:15:00,0.658768,0.859375,0.315678,0.804196,0.0,0.0
1991-07-15 23:30:00,0.658768,0.859375,0.283898,0.881119,0.0,0.0


## Define COBS environment

In [5]:
# Set E+ and IDF files paths 
Model.set_energyplus_folder("/usr/local/EnergyPlus-9-3-0/")
idf_files_path = "/mnt/c/users/Dave/Downloads/idf-sample-files/"

# Initialize the model with idf and weather files
# model = setup_env(idf_files_path+"2020/RefBldgLargeOfficeNew2004_Chicago.idf",
idf_path = idf_files_path+"2020/RefBldgMediumOfficeNew2004_Chicago.idf"
epw_path = "cobs/data/weathers/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw"

model = setup_env(idf_path, epw_path, season='summer', num_days=14, timestep=4, forecast_path=dataset_name)

In [6]:
agent_params = {
    'gamma': 0.99,
    'epsilon': 0.1,
    'lr': 0.0001,
    'batch_size': 64,
    'eps_min': 0.1,
    'eps_dec': 0.001,
    # Every 'replace' timestep we replace the network and the target network
    # In the paper found online, it replaces every 2 episodes, try it
    'replace': 24,
    'min_action': -5,
    'min_sat_action': 5,
    'max_action': 5,
    'max_sat_action': -5,
    # Need to keep distinct action values of [-20, 20], we round 1 decimal place
    'num_actions': 20,
    # Number of AHUs need to be implemented by actions
    'num_action_zones': 3,
    'mem_size': 240,
    'seed': 42
}

forecast_state = model.state_modifier.models[0].get_output_states()
    
obs = model.reset()
state, AHUs = vectorise(obs, dist_names, obs_history, state_names + forecast_state)
agent_AHU1 = DQNAgent({**agent_params, 'input_dims': (len(state) + len(AHUs[0]),)}, DQN_Network)
agent_AHU2 = DQNAgent({**agent_params, 'input_dims': (len(state) + len(AHUs[1]),)}, DQN_Network)
agent_AHU3 = DQNAgent({**agent_params, 'input_dims': (len(state) + len(AHUs[2]),)}, DQN_Network)

# num_episode = 100
# HVAC = np.zeros(num_episode)
# fans = np.zeros(num_episode)
# heating = np.zeros(num_episode)
# cooling = np.zeros(num_episode)
obs, actions, agent = run_dqn_episode(model, [agent_AHU1, agent_AHU2,agent_AHU3],
                                  dist_names, obs_history)
# for i in tqdm(range(num_episode)):
#     obs, actions, agent = run_dqn_episode(model, [agent_AHU1, agent_AHU2,agent_AHU3], dist_names, obs_history)
#     obs_history = pd.DataFrame(obs)
#     obs_history.index = obs_history['time']
#     obs_history.drop(columns=['time'], inplace=True)
#     HVAC[i] = np.sum(obs_history["HVAC Cost"]) / 4000
#     fans[i] = np.sum(obs_history["Fans Cost"]) / 4000
#     heating[i] = np.sum(obs_history["Heating Cost"]) / 4000
#     cooling[i] = np.sum(obs_history["Cooling Cost"]) / 4000
    
# plt.plot(HVAC)
# plt.show()
# plt.plot(fans)
# plt.show()
# plt.plot(heating)
# plt.show()
# plt.plot(cooling)
# plt.show()

EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2022.03.29 01:11
Initializing Response Factors
Calculating CTFs for "STEEL FRAME NON-RES EXT WALL", Construction # 1
Calculating CTFs for "IEAD NON-RES ROOF", Construction # 2
Calculating CTFs for "EXT-SLAB", Construction # 3
Calculating CTFs for "INT-WALLS", Construction # 4
Calculating CTFs for "INT-FLOOR-TOPSIDE", Construction # 5
Calculating CTFs for "DROPCEILING", Construction # 8
Calculating CTFs for "INTERIORFURNISHINGS", Construction # 9
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Wi

EnergyPlus Completed Successfully.


In [6]:
agent_params = {
    'gamma': 0.99,
    'epsilon': 0.1,
    'lr': 0.0001,
    'batch_size': 64,
    'eps_min': 0.1,
    'eps_dec': 0.001,
    # Every 'replace' timestep we replace the network and the target network
    # In the paper found online, it replaces every 2 episodes, try it
    'replace': 24,
    'min_action': -5,
    'min_sat_action': 5,
    'max_action': 5,
    'max_sat_action': -5,
    # Need to keep distinct action values of [-20, 20], we round 1 decimal place
    'num_actions': 20,
    # Number of AHUs need to be implemented by actions
    'num_action_zones': 3,
    'mem_size': 240,
    'seed': 42
}

forecast_state = model.state_modifier.models[0].get_output_states()
    
obs = model.reset()
state, AHUs = vectorise(obs, dist_names, obs_history, state_names + forecast_state)
bdqn_agent = BDDQNAgent({**agent_params, 'input_dims': (len(state) + np.array(AHUs).size, )}, BDDQN_Network)
obs, actions, agent = run_bdqn_episode(model, bdqn_agent, dist_names, obs_history)

EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2022.03.29 01:24
Initializing Response Factors
Calculating CTFs for "STEEL FRAME NON-RES EXT WALL", Construction # 1
Calculating CTFs for "IEAD NON-RES ROOF", Construction # 2
Calculating CTFs for "EXT-SLAB", Construction # 3
Calculating CTFs for "INT-WALLS", Construction # 4
Calculating CTFs for "INT-FLOOR-TOPSIDE", Construction # 5
Calculating CTFs for "DROPCEILING", Construction # 8
Calculating CTFs for "INTERIORFURNISHINGS", Construction # 9
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Wi

EnergyPlus Completed Successfully.


In [7]:
bdqn = pd.DataFrame(obs)
# dqn['time'] = dqn['time'].mask(dqn['time'].dt.year > 1,
#                              dqn['time'] + pd.offsets.DateOffset(year=1991))
bdqn.index = bdqn['time']
bdqn.drop(columns=['time'], inplace=True)
bdqn.head()

,timestep,temperature,occupancy,terminate,reward,PMV,PPD,Lights,Fans Cost,Cooling Cost,...,Ambient Temp. 7,Total Rad. 7,Ambient Temp. 8,Total Rad. 8,Ambient Temp. 9,Total Rad. 9,Ambient Temp. 10,Total Rad. 10,Ambient Temp. 11,Total Rad. 11
time,,,,,,,,,,,,,,,,,,,,,
1991-07-01 00:15:00,0,"{'Core_bottom': 24.637464688560218, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,NaN,"{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,0.236967,0.0,0.233412,0.0,0.229858,0.0,0.226303,0.0,0.222749,0.0
1991-07-01 00:30:00,1,"{'Core_bottom': 24.632549236026296, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23674295502753231, 'Core_mi...","{'Core_bottom': 6.163033888512302, 'Core_mid':...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,0.233412,0.0,0.229858,0.0,0.226303,0.0,0.222749,0.0,0.219194,0.0
1991-07-01 00:45:00,2,"{'Core_bottom': 24.627573422207853, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23754809813081051, 'Core_mi...","{'Core_bottom': 6.1709768620456344, 'Core_mid'...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,0.229858,0.0,0.226303,0.0,0.222749,0.0,0.219194,0.0,0.215640,0.0
1991-07-01 01:00:00,3,"{'Core_bottom': 24.622486445356554, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23838046384231054, 'Core_mi...","{'Core_bottom': 6.179217004316627, 'Core_mid':...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,0.226303,0.0,0.222749,0.0,0.219194,0.0,0.215640,0.0,0.212085,0.0
1991-07-01 01:15:00,4,"{'Core_bottom': 24.61727913492755, 'Core_mid':...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23924888109050627, 'Core_mi...","{'Core_bottom': 6.1878450497201385, 'Core_mid'...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,0.222749,0.0,0.219194,0.0,0.215640,0.0,0.212085,0.0,0.208531,0.0


In [7]:
dqn = pd.DataFrame(obs)
# dqn['time'] = dqn['time'].mask(dqn['time'].dt.year > 1,
#                              dqn['time'] + pd.offsets.DateOffset(year=1991))
dqn.index = dqn['time']
dqn.drop(columns=['time'], inplace=True)
dqn.head()

,timestep,temperature,occupancy,terminate,reward,PMV,PPD,Lights,Fans Cost,Cooling Cost,...,Ambient Temp. 7,Total Rad. 7,Ambient Temp. 8,Total Rad. 8,Ambient Temp. 9,Total Rad. 9,Ambient Temp. 10,Total Rad. 10,Ambient Temp. 11,Total Rad. 11
time,,,,,,,,,,,,,,,,,,,,,
1991-07-01 00:15:00,0,"{'Core_bottom': 24.637464688560218, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,NaN,"{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,0.236967,0.0,0.233412,0.0,0.229858,0.0,0.226303,0.0,0.222749,0.0
1991-07-01 00:30:00,1,"{'Core_bottom': 24.632549236026296, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23674295502753231, 'Core_mi...","{'Core_bottom': 6.163033888512302, 'Core_mid':...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,0.233412,0.0,0.229858,0.0,0.226303,0.0,0.222749,0.0,0.219194,0.0
1991-07-01 00:45:00,2,"{'Core_bottom': 24.627573422207853, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23754809813081051, 'Core_mi...","{'Core_bottom': 6.1709768620456344, 'Core_mid'...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,0.229858,0.0,0.226303,0.0,0.222749,0.0,0.219194,0.0,0.215640,0.0
1991-07-01 01:00:00,3,"{'Core_bottom': 24.622486445356554, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23838046384231054, 'Core_mi...","{'Core_bottom': 6.179217004316627, 'Core_mid':...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,0.226303,0.0,0.222749,0.0,0.219194,0.0,0.215640,0.0,0.212085,0.0
1991-07-01 01:15:00,4,"{'Core_bottom': 24.61727913492755, 'Core_mid':...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23924888109050627, 'Core_mi...","{'Core_bottom': 6.1878450497201385, 'Core_mid'...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,0.222749,0.0,0.219194,0.0,0.215640,0.0,0.212085,0.0,0.208531,0.0


In [62]:
# results_dir = 'simulation_results'
# dqn.to_pickle(os.path.join(results_dir, "DQN-100.pkl"))

# np.savetxt("DQN100HVACs.csv", HVAC, delimiter=",")
# np.savetxt("DQN100Fans.csv", fans, delimiter=",")
# np.savetxt("DQN100heatings.csv", heating, delimiter=",")
# np.savetxt("DQN100coolings.csv", cooling, delimiter=",")

## Compare Statistics and Plot

In [9]:
# Only get 7.1 - 7.14 for baseline
baseline = pd.read_pickle(dataset_name)[:1344]

In [10]:
def get_avg_PPD(df, working_time):
    # Get PPD only when in working hours and zone is occupied
    means = []
    for i in range(len(df)):
        curr = df.iloc[i]
        PPDs = []
        if curr.name.hour in working_time:
            for zone in curr['PPD'].keys():
                if curr['occupancy'][zone] > 0:
                    PPDs.append(curr['PPD'][zone])
        if len(PPDs) == 0:
            mean = 0
        else:
            mean = np.mean(PPDs)
        means.append(mean)
    return np.mean(means)


def getStatics(df, working_time):
    meanPPD = get_avg_PPD(df, working_time)
    print("Average PPD = ", meanPPD)
    
    fans_cost = np.sum(df["Fans Cost"]) * 900 / 1000 / 3600 # Unit in kWh
    print("Energy Consumed by the Fans in HVAC System = {:.2f}kWh".format(fans_cost))
    
    cooling_cost = np.sum(df["Cooling Cost"]) * 900 / 1000 / 3600 # Unit in kWh
    print("Energy Consumed by the Cooling in HVAC System = {:.2f}kWh".format(cooling_cost))
    
    heating_cost = np.sum(df["Heating Cost"]) * 900 / 1000 / 3600 # Unit in kWh
    print("Energy Consumed by the Heating in HVAC System = {:.2f}kWh".format(heating_cost))
    
    HVACEnergy = np.sum(df["HVAC Cost"]) * 900 / 1000 / 3600 # Unit in kWh
    print("Energy Consumed by the HVAC System = {:.2f}kWh".format(HVACEnergy))

In [11]:
working_time = range(8, 17)
print("**EnergyPlus Baseline**")
getStatics(baseline, working_time)
print("**DQN**")
getStatics(dqn, working_time)

**EnergyPlus Baseline**
Average PPD =  2.869365552228636
Energy Consumed by the Fans in HVAC System = 714868.68kWh
Energy Consumed by the Cooling in HVAC System = 6637262.63kWh
Energy Consumed by the Heating in HVAC System = 0.00kWh
Energy Consumed by the HVAC System = 7352131.32kWh
**DQN**
Average PPD =  10.576779182439639
Energy Consumed by the Fans in HVAC System = 4845854.02kWh
Energy Consumed by the Cooling in HVAC System = 1955020.48kWh
Energy Consumed by the Heating in HVAC System = 0.00kWh
Energy Consumed by the HVAC System = 6800874.51kWh


In [74]:
a = torch.tensor([100])
b = torch.tensor([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]).double()
(b - b.mean())[np.arange(64)]

IndexError: index 20 is out of bounds for dimension 0 with size 20

In [61]:
sat_actions_tups = [(3, 24.7), (5, 27.3), (-1, 22)]
# A_stpts = [1 * num_actions, 1 * num_actions, 1 * num_actions]
[t[1] for t in sat_actions_tups]

[24.7, 27.3, 22]

In [27]:
bqdn = BDDQN_Network(0.001, 44, 3, 20, '.').double()

In [28]:
bqdn.modules

<bound method Module.modules of BDDQN_Network(
  (linear1): Linear(in_features=44, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=256, bias=True)
  (linear3): Linear(in_features=256, out_features=128, bias=True)
  (V): Linear(in_features=128, out_features=1, bias=True)
  (A_stpt_0): Linear(in_features=128, out_features=20, bias=True)
  (A_stpt_1): Linear(in_features=128, out_features=20, bias=True)
  (A_stpt_2): Linear(in_features=128, out_features=20, bias=True)
  (loss): MSELoss()
)>

In [29]:
x = np.arange(44)

In [30]:
_, y = bqdn.forward(torch.tensor(x).double())

In [31]:
for a in y:
    idx = torch.argmax(a).item()
    print(idx)

1
10
8


In [49]:
agent_AHU1.choose_action((state + AHUs[0], model.reset(),0))

EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2022.03.29 00:12
Initializing Response Factors
Calculating CTFs for "STEEL FRAME NON-RES EXT WALL", Construction # 1
Calculating CTFs for "IEAD NON-RES ROOF", Construction # 2
Calculating CTFs for "EXT-SLAB", Construction # 3
Calculating CTFs for "INT-WALLS", Construction # 4
Calculating CTFs for "INT-FLOOR-TOPSIDE", Construction # 5
Calculating CTFs for "DROPCEILING", Construction # 8
Calculating CTFs for "INTERIORFURNISHINGS", Construction # 9
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Wi

(0.8, array([25.42275439, 26.11191068, 23.78932631]))

In [42]:
states, actions, rewards, states_, dones = agent_AHU1.sample_memory()
agent_AHU1.q_eval.forward(states)[np.arange(64), actions].shape

torch.Size([64, 1, 64])

In [7]:
bdqn_agent = BDDQNAgent({**agent_params, 'input_dims': (len(state) + np.array(AHUs).size, )}, BDDQN_Network, '.')

In [50]:
actions

tensor([[[ 2]],

        [[16]],

        [[16]],

        [[ 2]],

        [[18]],

        [[ 3]],

        [[ 0]],

        [[ 5]],

        [[10]],

        [[ 6]],

        [[15]],

        [[19]],

        [[15]],

        [[ 3]],

        [[15]],

        [[12]],

        [[ 3]],

        [[18]],

        [[16]],

        [[12]],

        [[ 4]],

        [[ 3]],

        [[ 3]],

        [[ 6]],

        [[ 5]],

        [[11]],

        [[ 2]],

        [[14]],

        [[ 7]],

        [[10]],

        [[13]],

        [[ 9]],

        [[ 6]],

        [[ 9]],

        [[14]],

        [[19]],

        [[ 2]],

        [[16]],

        [[19]],

        [[ 5]],

        [[13]],

        [[ 5]],

        [[ 0]],

        [[ 1]],

        [[18]],

        [[10]],

        [[ 3]],

        [[11]],

        [[12]],

        [[15]],

        [[ 4]],

        [[ 4]],

        [[ 1]],

        [[15]],

        [[ 2]],

        [[10]],

        [[ 9]],

        [[10]],

        [[11]]

In [60]:
state + AHUs[0] + AHUs[1] + AHUs[2]

[0.2012985200886601,
 0.9795299412524945,
 0.0,
 1.0,
 0.06540312923014306,
 0.9978589232386035,
 0.0010908305661643995,
 0.999999405044161,
 0.25829383886255924,
 0.0,
 0.2559241706161139,
 0.0,
 0.25355450236966837,
 0.0,
 0.2511848341232228,
 0.0,
 0.2476303317535546,
 0.0,
 0.2440758293838864,
 0.0,
 0.240521327014218,
 0.0,
 0.2369668246445498,
 0.0,
 0.23341232227488157,
 0.0,
 0.2298578199052132,
 0.0,
 0.22630331753554497,
 0.0,
 0.22274881516587677,
 0.0,
 0.25829383886255924,
 0.83203125,
 0.2754237288135593,
 0.972027972027972,
 0.0,
 0.0,
 0.7895418653937295,
 0.4927595527145584,
 0.3526648195386752,
 0.4631227542201513,
 0.348678769361551,
 0.38013800708505036,
 0.8639259395868674,
 0.6064355540852138,
 0.5291630705070145,
 0.5943093755960638,
 0.5214789989731954,
 0.4958305800633947,
 0.7065223553685874,
 0.49270242194267905,
 0.5470486266761125,
 0.49676386543012124,
 0.45233076675314743,
 0.2837669917639315]

In [15]:
{**agent_params, 'input_dims': (len(state) + 18, )}

{'gamma': 0.99,
 'epsilon': 0.1,
 'lr': 0.0001,
 'batch_size': 64,
 'eps_min': 0.1,
 'eps_dec': 0.001,
 'replace': 24,
 'min_action': -5,
 'min_sat_action': 5,
 'max_action': 5,
 'max_sat_action': -5,
 'num_actions': 20,
 'num_action_zones': 3,
 'mem_size': 240,
 'seed': 42,
 'input_dims': (56,)}

In [13]:
np.array(AHUs).size

18